<a href="https://colab.research.google.com/github/djDongjae/HongongML/blob/main/HongongML_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02067852, 0.01867533, 0.01462412, 0.02906489, 0.02992535]), 'score_time': array([0.00121427, 0.00121784, 0.00124884, 0.00120974, 0.0068717 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [12]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.001, 0.002, 0.003, 0.004, 0.005 ]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004,
                                                   0.005]})

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8724263998460651


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86165044 0.85761105 0.85607111 0.85414803 0.85318557]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.001}


In [19]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth':range(5,20,1), 'min_samples_split' : range(2, 100, 10)}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [23]:
#랜덤 서치
from scipy.stats import uniform, randint

In [24]:
rgen = randint(0, 10)
rgen.rvs(10)

array([1, 7, 8, 0, 8, 8, 7, 4, 0, 7])

In [25]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94, 113, 109,  94, 104,  99,  84,  95, 118,  90]))

In [26]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.14272443, 0.81280128, 0.32793776, 0.67986303, 0.51578024,
       0.10906693, 0.12956264, 0.56715853, 0.06778807, 0.95071506])

In [27]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), 'max_depth' : randint(20,50), 'min_samples_split' : randint(2, 25), 'min_samples_leaf': randint(1,25),}

In [28]:
from sklearn.model_selection import RandomizedSearchCV

In [30]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe8e1dc9790>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe8e1dc9c50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe8e21a1b90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe8e212c310>},
                   random_state=42)

In [31]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [32]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [33]:
dt = gs.best_estimator_

In [34]:
print(dt.score(test_input, test_target))

0.86
